In [20]:
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from nes_py.wrappers import JoypadSpace
from gym.wrappers import FrameStack, GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, VecTransposeImage
from matplotlib import pyplot as plt
import time

class CustomJoypadSpace(JoypadSpace):
    def reset(self, **kwargs):
        kwargs.pop('seed', None)
        kwargs.pop('options', None)
        return super().reset(**kwargs)

    def step(self, action):
        observation, reward, done, info = super().step(action)
        return observation, reward, done, info

class CustomDummyVecEnv(DummyVecEnv):
    def reset(self):
        for env_idx in range(self.num_envs):
            obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return self._obs_from_buf()

    def step_wait(self):
        results = [self.envs[env_idx].step(self.actions[env_idx]) for env_idx in range(self.num_envs)]
        for env_idx, (obs, reward, done, info) in enumerate(results):
            self.buf_rews[env_idx] = reward
            self.buf_dones[env_idx] = done
            self.buf_infos[env_idx] = info
            if done:
                obs = self.envs[env_idx].reset()
            self._save_obs(env_idx, obs)
        return self._obs_from_buf(), self.buf_rews, self.buf_dones, self.buf_infos

In [21]:
env = gym_super_mario_bros.make("SuperMarioBros-v0")
env = CustomJoypadSpace(env, SIMPLE_MOVEMENT)

env.render_mode = 'human'

c:\Users\ACER\Desktop\RL_in_gaming\marioenv\Lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3` with the environment ID `SuperMarioBros-v3`.
  logger.warn(


In [3]:
done = True

for step in range(1000): 
    if done: 
        env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()
env.close()

In [22]:
env = GrayScaleObservation(env, keep_dim=True)

env = CustomDummyVecEnv([lambda: env])

env = VecFrameStack(env, 4, channels_order='last')

env = VecTransposeImage(env)

c:\Users\ACER\Desktop\RL_in_gaming\marioenv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [23]:
import os

from stable_baselines3 import PPO

from stable_baselines3.common.callbacks import BaseCallback

In [24]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [25]:
CHECKPOINT_DIR = './train/'
log_dir ='./logs/'

In [26]:
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [27]:
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_dir, learning_rate=0.000001, n_steps=512, device="cuda")

Using cuda device


In [28]:
def callback(locals_, globals_):
    return locals_, globals_, True, None

In [29]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_23
----------------------------
| time/              |     |
|    fps             | 136 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 85            |
|    iterations           | 2             |
|    time_elapsed         | 11            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 1.1684839e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00417      |
|    learning_rate        | 1e-06         |
|    loss                 | 211           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000113     |
|    value_loss           | 516           |
------------------------

c:\Users\ACER\Desktop\RL_in_gaming\marioenv\Lib\site-packages\stable_baselines3\common\vec_env\stacked_observations.py:171: UserWarning: VecFrameStack wrapping a VecEnv without terminal_observation info
  warnings.warn("VecFrameStack wrapping a VecEnv without terminal_observation info")


ValueError: cannot step in a done environment! call `reset`

In [15]:
model.save('thisisatestmodel')

In [ ]:
model = PPO.load('C:/Users/ACER/Desktop/RL_in_gaming/thisisatestmodel')

In [ ]:
state = env.reset()

while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.01)
    if done:
        state = env.reset()